# Auto-generating Documentation

A blurb outlineing the notebook, and what we are doing

## Install Dependencies

In [2]:
!pip install replicate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


## Set Replicate Token

In [3]:
import os
from google.colab import userdata

os.environ['REPLICATE_API_TOKEN'] = userdata.get('REPLICATE_API_TOKEN')

## Define a function for downloading a repository

A blurb about the following function

`github_token` is optional, but if it is set you can access more endpoints and have a higher rate limit. More information here: https://docs.github.com/en/rest/authentication/authenticating-to-the-rest-api?apiVersion=2022-11-28

In [4]:
import requests
from time import sleep

def get_github_repo_contents(repo_url, directory_path, github_token = None):
    # Extracting the owner and repo name from the URL
    repo_parts = repo_url.rstrip('/').split('/')
    owner = repo_parts[-2]
    repo = repo_parts[-1]

    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{directory_path}"
    if github_token is not None:
      headers = {'Authorization': f'token {github_token}'}
      response = requests.get(api_url, headers = headers)
    else:
      response = requests.get(api_url)
    response.raise_for_status()

    contents = response.json()

    result = []

    for item in contents:
        if item['type'] == 'file':
            file_response = requests.get(item['download_url'])
            file_response.raise_for_status()
            file_content = file_response.text
            language = item['name'].split('.')[-1]
            if language == 'py':
                language = 'python'
            elif language == 'js':
                language = 'javascript'
            result.append(f"{item['path']}\n```{language}\n{file_content}\n```")
        elif item['type'] == 'dir':
            # Recursively go through subdirectories
            subdirectory_contents = get_github_repo_contents(repo_url, item['path'], github_token)
            result.append(subdirectory_contents)
        sleep(0.1)

    return "\n\n".join(result)


## Get code from `ibm-granite-community/utils`

Specify the `ibm_granite_community` directory because it has all of the functions.

In [17]:
prompt = get_github_repo_contents("https://github.com/ibm-granite-community/utils", "ibm_granite_community")

## Count the tokens

Our model has a context window of 128k, which means the _input_ plus the _output_ can be no longer than 128k.

In [36]:
from transformers import AutoTokenizer

model_path = "ibm-granite/granite-8B-Code-instruct-128k"
tokenizer = AutoTokenizer.from_pretrained(model_path)

print(f"Your git repo load has {len(tokenizer(prompt, return_tensors='pt')['input_ids'][0])} tokens")

Your git repo load has 714 tokens


### Create our prompt and call the model in Replicate

Blurb about the following

In [24]:
import replicate

full_prompt = prompt + """

Provide detailed developer documentation for each function provided above.

Response Template:
## `function_name`

* _param1_: (type) description"

Synopsis of the function

_**returns**_:
"""

output = replicate.run(
    "ibm-granite/granite-8b-code-instruct-128k",
    input={

        "prompt": full_prompt,
        "max_tokens": 10000,
        "min_tokens": 0,
        "temperature": 0.75,
        "system_prompt": "You are a helpful assistant.",
        "presence_penalty": 0,
        "frequency_penalty": 0
    })


print("".join(output))

## `find_langchain_model`

* `platform`: (string) The platform to use for the model, either "replicate" or "ollama".
* `model_id`: (string) The ID of the model to use.
* `model_kwargs`: (dict) Additional keyword arguments to pass to the model constructor.

Returns:
- The loaded model object.

Synopsis:
The `find_langchain_model` function is used to load a model from either the Replicate or Ollama platform. It takes in the platform, model ID, and any additional keyword arguments for the model constructor.

Example:
```python
model = find_langchain_model("replicate", "text-davinci-003", temperature=0.8)
```

## `find_langchain_vector_db`

* `provider`: (string) The provider of the vector database, either "milvus" or "chroma".
* `embeddings_model`: (function) The function to use for embeddings.
* `model_kwargs`: (dict) Additional keyword arguments to pass to the vector database constructor.

Returns:
- The loaded vector database object.

Synopsis:
The `find_langchain_vector_db` function i